In [1]:
import pylab
from pycbc import waveform

In [3]:
waveform.print_fd_approximants()

LalSimulation Approximants
  EccentricFD
  TaylorF2
  TaylorF2Ecc
  TaylorF2NLTides
  TaylorF2RedSpin
  TaylorF2RedSpinTidal
  SpinTaylorF2
  EOBNRv2_ROM
  EOBNRv2HM_ROM
  SEOBNRv1_ROM_EffectiveSpin
  SEOBNRv1_ROM_DoubleSpin
  SEOBNRv2_ROM_EffectiveSpin
  SEOBNRv2_ROM_DoubleSpin
  SEOBNRv2_ROM_DoubleSpin_HI
  Lackey_Tidal_2013_SEOBNRv2_ROM
  SEOBNRv4_ROM
  SEOBNRv4HM_ROM
  SEOBNRv4_ROM_NRTidal
  SEOBNRv4_ROM_NRTidalv2
  SEOBNRv4_ROM_NRTidalv2_NSBH
  SEOBNRv4T_surrogate
  IMRPhenomA
  IMRPhenomB
  IMRPhenomC
  IMRPhenomD
  IMRPhenomD_NRTidal
  IMRPhenomD_NRTidalv2
  IMRPhenomNSBH
  IMRPhenomHM
  IMRPhenomP
  IMRPhenomPv2
  IMRPhenomPv2_NRTidal
  IMRPhenomPv2_NRTidalv2
  SpinTaylorT4Fourier
  SpinTaylorT5Fourier
  NRSur4d2s
  IMRPhenomXAS
  IMRPhenomXHM
  IMRPhenomPv3
  IMRPhenomPv3HM
  IMRPhenomXP
  IMRPhenomXPHM
  SpinTaylorF2_SWAPPER
  TaylorF2NL
  PreTaylorF2
  multiband
  TaylorF2e
  TaylorF2_INTERP
  SpinTaylorT5
  SEOBNRv1_ROM_EffectiveSpin_INTERP
  SEOBNRv1_ROM_DoubleSpin_INTERP


In [ ]:
from pycbc.waveform import get_fd_waveform
import numpy as np
import pycbc.conversions as conversions
import lal

def IMRPhenomdipole(B=0.0, **kwds):
    if 'approximant' in kwds:
        kwds.pop("approximant")
    hp, hc = get_fd_waveform(approximant="IMRPhenomXPHM", **kwds)

    eta = conversions.eta_from_mass1_mass2(kwds['mass1'],kwds['mass2'])
    M_chirp = conversions.mchirp_from_mass1_mass2(kwds['mass1'],kwds['mass2'])
    M = conversions.mtotal_from_mass1_mass2(kwds['mass1'],kwds['mass2'])

    
    kmin = int(kwds['f_lower']/kwds['delta_f'])
    fsampling = hp.sample_frequencies[kmin:]

    beta = -3/224*eta**(2/5)*B
    dipole = np.exp(1j*beta*(np.pi*M_chirp*fsampling*lal.MTSUN_SI)**(-7/3))
    #dipole[(fsampling>fISCO)] = np.ones_like(fsampling[(fsampling>fISCO)])
    
    hp[kmin:], hc[kmin:] = hp[kmin:]*dipole, hc[kmin:]*dipole

    #derive_B = (-1j) * (3/(224 * eta)) * (pi*M*fsampling*s_g)**(-7/3) * hcd

    return hp,hc

In [ ]:
# Now we can add this waveform to pycbc directly. This will make the waveform 
# accessible from get_fd_waveform and higher level functions.
from pycbc.waveform import add_custom_waveform

# To protect against overriding an existing waveform, you'll get an error if you run
# this function twice, to override this, you can set force=True.
# add_custom_waveform('my_waveform', my_waveform, 'frequency')
add_custom_waveform('IMRPhenomdipole', IMRPhenomdipole, 'frequency', force=True)

In [ ]:
for B in [0,1e-2]:
    hp, hc = get_fd_waveform(approximant="IMRPhenomdipole", B=B,
                         mass1=30, mass2=30, 
                         f_lower=30, delta_f=1.0/64)
    pylab.plot(hp.sample_frequencies, hp.real(),label=str(B))

    
hpdef, hcdef = get_fd_waveform(approximant="IMRPhenomXPHM",
                         mass1=30, mass2=30, 
                         f_lower=30, delta_f=1.0/64)
pylab.plot(hpdef.sample_frequencies,hpdef.real(),label='XPHM')
pylab.xscale('log')
pylab.xlim(30, 50)
pylab.xlabel('Frequency (Hz)')
pylab.ylabel('Strain')
pylab.legend()